In [1]:
import bempp.api 
import numpy as np
import mtf
from mtf.utils import bhmie
from mtf.config import config
from matplotlib import pyplot as plt
from decimal import Decimal

bempp.api.enable_console_logging()
M = 2

mtf.config.set_case("B")

#k_ext, k_int = config["k_ext"], config["k_int"]
k0, k1 = config["k_ext"], config["k_int"]
lambda_par, freq = config["lambda"], config["frequency"]

polarization = config["polarization"]
direction = config["direction"]

eps_rel = config["eps_rel"]
mu_rel = config["mu_rel"]
mu0 = config["mu_ext"]
mu1 = mu_rel * mu0

eta_rel = np.sqrt(mu_rel / eps_rel)

print("The exterior wavenumber is: {0}".format(k0))
print("The interior wavenumber is: {0}".format(k1))

print("----")
print("The exterior wavelenght is: {0}".format(lambda_par))
print("The exterior frequency is: {:.2E}".format(Decimal(freq)))

#k0 = 10
#mu0 = 1
#refractivei = [1.5, 1.2]
#mui = [1., 1.]
precision = 1

h = 2 * np.pi/(precision*k0)
print(h, ': h')
print(precision, ': precision')

The exterior wavenumber is: 5.0
The interior wavenumber is: 6.892024376045111
----
The exterior wavelenght is: 1.2566370614359172
The exterior frequency is: 2.39E+8
1.2566370614359172 : h
1 : precision


In [2]:
segments = [[10], [10]]
swapped_normals = [[10], []]

grid = bempp.api.shapes.sphere(h=h)
k_int, k_ext = config["k_int"], config["k_ext"]
print(grid.number_of_edges)
n = k_int / k_ext
refIndex = n
numAngles = 901
s1, s2, qext, qsca, qback, gsca = bhmie(k_ext, k_int / k_ext, numAngles)
#transmission_operators = assemble_operators(grid, config)
#far_field, solution = evaluate_far_field(transmission_operators, config)


bempp:HOST:INFO: Created grid with id da0d3989-530c-482c-8092-c9f6e14e0da2. Elements: 40. Edges: 60. Vertices: 22


60


In [3]:
k_list = [k0]
eta_rel_list = [1]
mu_list = [mu0]

for index in range(M-1):
  k_list.append(k1)
  mu_list.append(mu1)
  eta_rel_list.append(eta_rel)
    
dA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
rA = [bempp.api.function_space(grid, "RWG", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]
tA = [bempp.api.function_space(grid, "SNC", 0, segments=seg, swapped_normals=normals,
                                      include_boundary_dofs=True)
              for seg, normals in zip(segments, swapped_normals)]

In [4]:
multitrace_ops_fosd = []
multitrace_ops_ak = []

# > Assemble all diagonal operators
for index in range(M):
  k = k_list[index]
  mu = mu_list[index]
  eta = eta_rel_list[index]
  efie = bempp.api.operators.boundary.maxwell.electric_field(dA[1], rA[1], tA[1], k)
  mfie = bempp.api.operators.boundary.maxwell.magnetic_field(dA[1], rA[1], tA[1], k)
  multitrace_ops_fosd.append(bempp.api.GeneralizedBlockedOperator([[mfie, eta * efie],[- 1/eta * efie, mfie]]))
  multitrace_ops_ak.append(bempp.api.GeneralizedBlockedOperator([[mfie, k / mu * efie],[- mu /k * efie, mfie]]))


In [5]:
lhs_op_fosd = multitrace_ops_fosd[0] + multitrace_ops_fosd[1]
lhs_op_ak = multitrace_ops_ak[0] + multitrace_ops_ak[1]

In [6]:
A_fosd = bempp.api.as_matrix(lhs_op_fosd.weak_form())
A_ak = bempp.api.as_matrix(lhs_op_ak.weak_form())

bempp:HOST:INFO: OpenCL CPU Device set to: pthread-AMD EPYC 7302 16-Core Processor


In [7]:
print(np.linalg.cond(A_fosd), 'cond_2 FOSD')
print(np.linalg.cond(A_ak), 'cond_2 AK')

29.317485544969003 cond_2 FOSD
418328586376000.4 cond_2 AK
